# Qwen3 RAG System - Complete Implementation

A production-ready RAG (Retrieval-Augmented Generation) system using Qwen3 models with full answer generation capabilities.

## Features
- **PDF document processing** with intelligent chunking
- **Qwen3 embeddings** for semantic search and retrieval
- **Qwen3 reranker** for precision improvement
- **Qwen3 LLM** for answer generation (true RAG!)
- **ChromaDB vector store** with persistence
- **Clean, modular architecture** following RAG best practices
- **Complete pipeline**: Document → Embedding → Retrieval → Reranking → Generation

## RAG Pipeline
1. **Document Processing**: Load and chunk PDF documents
2. **Embedding**: Convert text to vectors using Qwen3-Embedding
3. **Storage**: Store in ChromaDB vector database
4. **Retrieval**: Find relevant documents using semantic similarity
5. **Reranking**: Improve precision with Qwen3-Reranker
6. **Generation**: Generate natural language answers with Qwen3 LLM

## Installation

In [1]:
!pip install transformers sentence-transformers vllm flash-attn chromadb PyPDF2

## Import and Setup

In [3]:
from qwen3_rag import Qwen3RAG, RAGConfig
import logging
import os

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Configuration

In [4]:
# Configure RAG system
config = RAGConfig(
    chunk_size=512,
    chunk_overlap=50,
    top_k_retrieval=20,
    top_k_rerank=5,
    similarity_threshold=0.3,
    device="cuda:0",
    collection_name="my_documents",
    persist_directory="./my_chroma_db"
)

print(f"Configuration: {config}")

Configuration: RAGConfig(embedding_model='Qwen/Qwen3-Embedding-0.6B', reranker_model='Qwen/Qwen3-Reranker-0.6B', generator_model='Qwen/Qwen2.5-1.5B-Instruct', chunk_size=512, chunk_overlap=50, top_k_retrieval=20, top_k_rerank=5, similarity_threshold=0.3, device='cuda:0', collection_name='my_documents', persist_directory='./my_chroma_db', max_context_length=4000, generation_temperature=0.1, generation_max_tokens=512)


## Initialize RAG System

In [5]:
# Configure RAG system with generation model
config = RAGConfig(
    chunk_size=512,
    chunk_overlap=50,
    top_k_retrieval=20,
    top_k_rerank=5,
    similarity_threshold=0.3,
    device="cuda:0",
    collection_name="my_documents",
    persist_directory="./my_chroma_db",
    generator_model="Qwen/Qwen2.5-3B-Instruct",  # You can change this to any supported model
    generation_temperature=0.1,
    generation_max_tokens=2048
)

# Test different query modes
queries = [
    "What is  Amazon’s total revenue grew in 2023?",
    "What are the areas of focus in 2024 for Amazon?",
    "How is the Amazon’s Advertising progress?"
]

# Test retrieval only (no answer generation)
rag = Qwen3RAG(config)
print("=== Query Results (Retrieval Only) ===")
for query in queries[:2]:  # Test first 2 queries
    print(f"\nQuery: {query}")
    results = rag.query(query, use_reranker=True)
    
    print(f"Found {len(results['documents'])} relevant documents")
    for i, (doc, sim, rerank, meta) in enumerate(zip(
        results['documents'], 
        results['similarities'], 
        results['rerank_scores'],
        results['metadatas']
    )):
        source = meta.get('source', 'Unknown')
        print(f"  {i+1}. [Similarity: {sim:.3f}] [Rerank: {rerank:.3f}] [Source: {source}]")
        print(f"     {doc[:150]}...\" if len(doc) > 150 else f\"     {doc}")
    print("-" * 80)

INFO:qwen3_rag:Loading embedding model: Qwen/Qwen3-Embedding-0.6B
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda:0
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: Qwen/Qwen3-Embedding-0.6B
INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
INFO:sentence_transformers.SentenceTransformer:1 prompt is loaded, with the key: query
INFO:qwen3_rag:Loaded existing collection: my_documents
INFO:qwen3_rag:Processing query: What is  Amazon’s total revenue grew in 2023?


=== Query Results (Retrieval Only) ===

Query: What is  Amazon’s total revenue grew in 2023?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:qwen3_rag:Loading reranker model: Qwen/Qwen3-Reranker-0.6B
INFO:qwen3_rag:Reranking documents...
INFO:qwen3_rag:Processing query: What are the areas of focus in 2024 for Amazon?


Found 5 relevant documents
  1. [Similarity: 0.739] [Rerank: 0.999] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     --- Page 1 --- Dear Shareholders: Last year at this time, I shared my enthusiasm and optimism for Amazon’s future. Today, I have even more. The reason..." if len(doc) > 150 else f"     --- Page 1 --- Dear Shareholders: Last year at this time, I shared my enthusiasm and optimism for Amazon’s future. Today, I have even more. The reasons are many, but start with the progress we’ve made in our financial results and customerexperiences, and extend to our continued innovation and the remarkable opportunities in front of us. In 2023, Amazon’s total revenue grew 12% year-over-year (“Y oY”) from $514B to $575B. By segment, North America revenue increased 12% Y oY from $316B to $353B, International revenue grew 11% Y oY from$118B to $131B, and AWS revenue increased 13% Y oY from $80B to $91B. Further, Amazon’s operating income and Free Cash Flow (“FCF”) dramaticall

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:qwen3_rag:Reranking documents...


Found 5 relevant documents
  1. [Similarity: 0.569] [Rerank: 0.996] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     everyday essentials business is growing (over 20% Y oY in Q4 2023). Our regionalization efforts have also trimmed transportation distances, helping lo..." if len(doc) > 150 else f"     everyday essentials business is growing (over 20% Y oY in Q4 2023). Our regionalization efforts have also trimmed transportation distances, helping lower our cost to serve. In 2023, for the first time since 2018, we reduced our cost to serve on a per unit basis globally. In the U.S. alone, cost to serve was down by more than $0.45 per unit Y oY . Decreasing cost to serve allows us both to investin speed improvements and afford adding more selection at lower Average Selling Prices (“ASPs”). Moreselection at lower prices puts us in consideration for more purchases. As we look toward 2024 (and beyond), we’re not done lowering our cost to serve. We’ve challenged every closely h

In [6]:
# RAG Answer Generation - This is true RAG!
print("=== RAG Answer Generation ===\n")
for query in queries[:3]:  # Test first 3 queries
    print(f"\nQuestion: {query}")
    print("=" * 60)
    
    # Get complete RAG answer
    result = rag.answer(query, use_reranker=True)
    
    print(f"Answer: {result['answer']}")
    
    print(f"\nSources used:")
    for i, (source, citation) in enumerate(zip(result['sources'][:3], result['citations'][:3])):  # Show top 3 sources
        print(f"  {i+1}. {citation}")
        print(f"     Chunk: {source['chunk_id']}")
    
    print(f"\nFull Context Citations:")
    for citation in result['citations']:
        print(f"  • {citation}")
    
    print("-" * 80)

INFO:qwen3_rag:Generating RAG answer for: What is  Amazon’s total revenue grew in 2023?
INFO:qwen3_rag:Processing query: What is  Amazon’s total revenue grew in 2023?


=== RAG Answer Generation ===


Question: What is  Amazon’s total revenue grew in 2023?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:qwen3_rag:Reranking documents...
INFO:qwen3_rag:Context truncated at 1 documents due to length limit
INFO:qwen3_rag:Cleaning up reranker to free memory for generator...
INFO:qwen3_rag:Reranker cleaned up successfully
INFO:qwen3_rag:Loading generator model...
INFO:qwen3_rag:Loading generator model: Qwen/Qwen2.5-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Answer: In 2024, the areas of focus for Amazon include:

1. Inbound fulfillment architecture and resulting inventory placement.
2. International expansion, particularly in emerging geographies such as India, Brazil, Australia, Mexico, Middle East, and Africa.

These areas are being evaluated with the aim of finding ways to reduce costs further while still improving customer service. [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf, Chunk: 1]

Sources used:
  1. [Source: Amazon-com-Inc-2023-Shareholder-Letter.pdf, Page 11]
     Chunk: 2
  2. [Source: Amazon-com-Inc-2023-Shareholder-Letter.pdf, Page 11]
     Chunk: 0
  3. [Source: Amazon-com-Inc-2023-Shareholder-Letter.pdf, Page 11]
     Chunk: 1

Full Context Citations:
  • [Source: Amazon-com-Inc-2023-Shareholder-Letter.pdf, Page 11]
  • [Source: Amazon-com-Inc-2023-Shareholder-Letter.pdf, Page 11]
  • [Source: Amazon-com-Inc-2023-Shareholder-Letter.pdf, Page 11]
  • [Source: Amazon-com-Inc-2023-Shareholder-Letter.pdf, Page 1

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:qwen3_rag:Loading reranker model: Qwen/Qwen3-Reranker-0.6B
INFO:qwen3_rag:Reranking documents...
INFO:qwen3_rag:Context truncated at 1 documents due to length limit
INFO:qwen3_rag:Cleaning up reranker to free memory for generator...
INFO:qwen3_rag:Reranker cleaned up successfully
INFO:qwen3_rag:Processing query: How is the Amazon’s Advertising progress?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer: According to the context, Amazon's Advertising progress has been strong, growing 24% year-over-year from $38 billion in 2022 to $47 billion in 2023. The growth was primarily driven by their sponsored ads. Additionally, they recently introduced Sponsored TV, which is a self-service solution for brands to create campaigns that can appear on up to 30+ streaming TV services, including Amazon Freevee and Twitch. They have also expanded their streaming TV advertising by incorporating ads into Prime Video shows and movies, targeting over 200 million monthly viewers in their most popular entertainment offerings.

Sources used:
  1. [Source: Amazon-com-Inc-2023-Shareholder-Letter.pdf, Page 11]
     Chunk: 1
  2. [Source: Amazon-com-Inc-2023-Shareholder-Letter.pdf, Page 11]
     Chunk: 14
  3. [Source: Amazon-com-Inc-2023-Shareholder-Letter.pdf, Page 11]
     Chunk: 0

Full Context Citations:
  • [Source: Amazon-com-Inc-2023-Shareholder-Letter.pdf, Page 11]
  • [Source: Amazon-com-Inc-20

In [7]:
# Simple Chat Interface
print("=== Simple Chat Interface ===\n")

# Test simple chat responses
test_questions = [
    "What is Amazon’s total revenue grew in 2023?",
    "What are the areas of focus in 2024 for Amazon?",
    "How is the Amazon’s Advertising progress?",
]

for question in test_questions:
    print(f"\nQ: {question}")
    answer = rag.chat(question, use_reranker=True)
    print(f"A: {answer}")
    print("-" * 50)

INFO:qwen3_rag:Generating RAG answer for: What is Amazon’s total revenue grew in 2023?
INFO:qwen3_rag:Processing query: What is Amazon’s total revenue grew in 2023?


=== Simple Chat Interface ===


Q: What is Amazon’s total revenue grew in 2023?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:qwen3_rag:Loading reranker model: Qwen/Qwen3-Reranker-0.6B
INFO:qwen3_rag:Reranking documents...
INFO:qwen3_rag:Context truncated at 1 documents due to length limit
INFO:qwen3_rag:Cleaning up reranker to free memory for generator...
INFO:qwen3_rag:Reranker cleaned up successfully
INFO:qwen3_rag:Processing query: What is Amazon’s total revenue grew in 2023?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

A: In 2024, Amazon's areas of focus include improving their inbound fulfillment architecture and inventory placement. These areas are believed to hold potential for further cost reductions while still ensuring fast service delivery for customers. [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf, Chunk: 1]
--------------------------------------------------

Q: How is the Amazon’s Advertising progress?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:qwen3_rag:Loading reranker model: Qwen/Qwen3-Reranker-0.6B
INFO:qwen3_rag:Reranking documents...
INFO:qwen3_rag:Context truncated at 1 documents due to length limit
INFO:qwen3_rag:Cleaning up reranker to free memory for generator...
INFO:qwen3_rag:Reranker cleaned up successfully
INFO:qwen3_rag:Processing query: How is the Amazon’s Advertising progress?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

A: According to the context, Amazon's Advertising progress has been strong. Specifically, it states that Amazon's Advertising grew 24% year-over-year from $38 billion in 2022 to $47 billion in 2023. The growth was primarily driven by their sponsored ads. Additionally, they recently introduced Sponsored TV, which is a self-service solution for brands to create campaigns that can appear on up to 30+ streaming TV services, including Amazon Freevee and Twitch. They also expanded their streaming TV advertising by incorporating ads into Prime Video shows and movies, targeting over 200 million monthly viewers in their most popular entertainment offerings.
--------------------------------------------------


## Simple Interface

In [8]:
def interactive_rag_chat():
    """Interactive RAG chat interface with full answer generation"""
    print("Interactive RAG Chat Interface")
    print("Type 'quit' to exit, 'sources' to see sources for last answer, 'citations' to see citations")
    print("-" * 50)
    
    last_result = None
    
    while True:
        question = input("\nAsk me anything: ").strip()
        
        if question.lower() in ['quit', 'exit', 'q']:
            break
            
        if question.lower() == 'sources' and last_result:
            print("\nDetailed Sources for last answer:")
            for i, source in enumerate(last_result['sources']):
                print(f"  {i+1}. Source: {source['source']}")
                print(f"     Chunk: {source['chunk_id']}")
                if source['page']:
                    print(f"     Page: {source['page']}")
            continue
            
        if question.lower() == 'citations' and last_result:
            print("\nCitations for last answer:")
            for citation in last_result['citations']:
                print(f"  • {citation}")
            continue
            
        if not question:
            continue
            
        try:
            # Get RAG answer
            print("\n🤖 Thinking...")
            last_result = rag.answer(question, use_reranker=True)
            
            print(f"\n✅ {last_result['answer']}")
            
            if last_result['sources']:
                print(f"\n📚 Based on {len(last_result['sources'])} source(s):")
                for citation in last_result['citations'][:3]:  # Show top 3 citations
                    print(f"  • {citation}")
                if len(last_result['citations']) > 3:
                    print(f"  ... and {len(last_result['citations']) - 3} more sources")
                print("\nType 'sources' for detailed info or 'citations' for all citations.")
                
        except Exception as e:
            print(f"❌ Error: {e}")
    
    print("👋 Goodbye!")

# Uncomment to run interactive RAG chat
# interactive_rag_chat()

## Query without Reranker

In [9]:
# Example queries
queries = [
    "What is Amazon’s total revenue grew in 2023?",
    "What are the areas of focus in 2024 for Amazon?",
    "How is the Amazon’s Advertising progress?"
]

# Test query without reranker
print("=== Query Results (Embedding Only) ===")
for query in queries[:2]:  # Test first 2 queries
    print(f"\nQuery: {query}")
    results = rag.query(query, use_reranker=False)
    
    print(f"Found {len(results['documents'])} relevant documents")
    for i, (doc, sim, meta) in enumerate(zip(results['documents'], results['similarities'], results['metadatas'])):
        source = meta.get('source', 'Unknown')
        print(f"  {i+1}. [Similarity: {sim:.3f}] [Source: {source}]")
        print(f"     {doc[:150]}..." if len(doc) > 150 else f"     {doc}")
    print("-" * 80)

INFO:qwen3_rag:Processing query: What is Amazon’s total revenue grew in 2023?


=== Query Results (Embedding Only) ===

Query: What is Amazon’s total revenue grew in 2023?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:qwen3_rag:Processing query: What are the areas of focus in 2024 for Amazon?


Found 13 relevant documents
  1. [Similarity: 0.741] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     --- Page 1 --- Dear Shareholders: Last year at this time, I shared my enthusiasm and optimism for Amazon’s future. Today, I have even more. The reason...
  2. [Similarity: 0.582] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     appreciated,and should bode well for customers and AWS longer-term. By the end of 2023, we saw cost optimizationattenuating, new deals accelerating, c...
  3. [Similarity: 0.566] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     everyday essentials business is growing (over 20% Y oY in Q4 2023). Our regionalization efforts have also trimmed transportation distances, helping lo...
  4. [Similarity: 0.503] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     with a cloudcomputing business at nearly a $100B revenue run rate, more than 85% of the global IT spend is still --- Page 8 --- on-premises. These bus

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Found 15 relevant documents
  1. [Similarity: 0.599] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     appreciated,and should bode well for customers and AWS longer-term. By the end of 2023, we saw cost optimizationattenuating, new deals accelerating, c...
  2. [Similarity: 0.572] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     --- Page 1 --- Dear Shareholders: Last year at this time, I shared my enthusiasm and optimism for Amazon’s future. Today, I have even more. The reason...
  3. [Similarity: 0.569] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     everyday essentials business is growing (over 20% Y oY in Q4 2023). Our regionalization efforts have also trimmed transportation distances, helping lo...
  4. [Similarity: 0.530] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     to our initial public offering in May 1997 and our $75 million loan, affording us substantial strategic flexibility. Our Employees The past year’s suc

## Query with Reranker

In [10]:
# Test query with reranker
print("=== Query Results (With Reranker) ===")
for query in queries:  # Test first 2 queries
    print(f"\nQuery: {query}")
    results = rag.query(query, use_reranker=True)
    
    print(f"Found {len(results['documents'])} relevant documents")
    for i, (doc, sim, rerank, meta) in enumerate(zip(
        results['documents'], 
        results['similarities'], 
        results['rerank_scores'],
        results['metadatas']
    )):
        source = meta.get('source', 'Unknown')
        print(f"  {i+1}. [Similarity: {sim:.3f}] [Rerank: {rerank:.3f}] [Source: {source}]")
        print(f"     {doc[:150]}..." if len(doc) > 150 else f"     {doc}")
    print("-" * 80)

INFO:qwen3_rag:Processing query: What is Amazon’s total revenue grew in 2023?


=== Query Results (With Reranker) ===

Query: What is Amazon’s total revenue grew in 2023?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:qwen3_rag:Loading reranker model: Qwen/Qwen3-Reranker-0.6B
INFO:qwen3_rag:Reranking documents...
INFO:qwen3_rag:Processing query: What are the areas of focus in 2024 for Amazon?


Found 5 relevant documents
  1. [Similarity: 0.741] [Rerank: 0.999] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     --- Page 1 --- Dear Shareholders: Last year at this time, I shared my enthusiasm and optimism for Amazon’s future. Today, I have even more. The reason...
  2. [Similarity: 0.566] [Rerank: 0.983] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     everyday essentials business is growing (over 20% Y oY in Q4 2023). Our regionalization efforts have also trimmed transportation distances, helping lo...
  3. [Similarity: 0.503] [Rerank: 0.756] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     with a cloudcomputing business at nearly a $100B revenue run rate, more than 85% of the global IT spend is still --- Page 8 --- on-premises. These bus...
  4. [Similarity: 0.582] [Rerank: 0.488] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     appreciated,and should bode well for customers and AWS longer-term. By the end of 2023,

...

INFO:qwen3_rag:Reranking documents...
INFO:qwen3_rag:Processing query: How is the Amazon’s Advertising progress?


Found 5 relevant documents
  1. [Similarity: 0.569] [Rerank: 0.996] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     everyday essentials business is growing (over 20% Y oY in Q4 2023). Our regionalization efforts have also trimmed transportation distances, helping lo...
  2. [Similarity: 0.599] [Rerank: 0.967] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     appreciated,and should bode well for customers and AWS longer-term. By the end of 2023, we saw cost optimizationattenuating, new deals accelerating, c...
  3. [Similarity: 0.572] [Rerank: 0.956] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     --- Page 1 --- Dear Shareholders: Last year at this time, I shared my enthusiasm and optimism for Amazon’s future. Today, I have even more. The reason...
  4. [Similarity: 0.503] [Rerank: 0.657] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     stack is the application layer. We’re building a substantial number of GenAI applicatio

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:qwen3_rag:Reranking documents...


Found 5 relevant documents
  1. [Similarity: 0.547] [Rerank: 0.997] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     everyday essentials business is growing (over 20% Y oY in Q4 2023). Our regionalization efforts have also trimmed transportation distances, helping lo...
  2. [Similarity: 0.457] [Rerank: 0.988] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     --- Page 1 --- Dear Shareholders: Last year at this time, I shared my enthusiasm and optimism for Amazon’s future. Today, I have even more. The reason...
  3. [Similarity: 0.400] [Rerank: 0.887] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     appreciated,and should bode well for customers and AWS longer-term. By the end of 2023, we saw cost optimizationattenuating, new deals accelerating, c...
  4. [Similarity: 0.423] [Rerank: 0.789] [Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf]
     with a cloudcomputing business at nearly a $100B revenue run rate, more than 85% of the

## Get Formatted Context

In [11]:
# Get formatted context for use with LLM
query = "What is Amazon’s total revenue grew in 2023?"
context = rag.get_context(query, use_reranker=True)

print(f"Query: {query}")
print("\nContext for LLM:")
print("=" * 50)
print(context)
print("=" * 50)

INFO:qwen3_rag:Processing query: What is Amazon’s total revenue grew in 2023?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:qwen3_rag:Reranking documents...
INFO:qwen3_rag:Context truncated at 1 documents due to length limit


Query: What is Amazon’s total revenue grew in 2023?

Context for LLM:
[Source: ./data/Amazon-com-Inc-2023-Shareholder-Letter.pdf, Chunk: 0]
--- Page 1 --- Dear Shareholders: Last year at this time, I shared my enthusiasm and optimism for Amazon’s future. Today, I have even more. The reasons are many, but start with the progress we’ve made in our financial results and customerexperiences, and extend to our continued innovation and the remarkable opportunities in front of us. In 2023, Amazon’s total revenue grew 12% year-over-year (“Y oY”) from $514B to $575B. By segment, North America revenue increased 12% Y oY from $316B to $353B, International revenue grew 11% Y oY from$118B to $131B, and AWS revenue increased 13% Y oY from $80B to $91B. Further, Amazon’s operating income and Free Cash Flow (“FCF”) dramatically improved. Operating income in 2023 improved 201% Y oY from $12.2B (an operating margin of 2.4%) to $36.9B (an operatingmargin of 6.4%). Trailing Twelve Month FCF adjusted for e

## Interactive Query Interface

In [12]:
def interactive_query():
    """Interactive query interface"""
    print("Interactive RAG Query Interface")
    print("Type 'quit' to exit")
    print("-" * 40)
    
    while True:
        query = input("\nEnter your question: ").strip()
        
        if query.lower() in ['quit', 'exit', 'q']:
            break
            
        if not query:
            continue
            
        try:
            # Get results with reranker
            results = rag.query(query, use_reranker=True)
            
            if not results['documents']:
                print("No relevant documents found.")
                continue
                
            print(f"\nFound {len(results['documents'])} relevant documents:")
            for i, (doc, rerank, meta) in enumerate(zip(
                results['documents'][:3],  # Show top 3
                results['rerank_scores'][:3],
                results['metadatas'][:3]
            )):
                source = meta.get('source', 'Unknown')
                print(f"\n{i+1}. [Relevance: {rerank:.3f}] [Source: {source}]")
                print(f"   {doc[:200]}..." if len(doc) > 200 else f"   {doc}")
                
        except Exception as e:
            print(f"Error processing query: {e}")
    
    print("Goodbye!")

# Uncomment to run interactive interface
# interactive_query()

## Cleanup Resources

In [13]:
# Cleanup when done
try:
    rag.cleanup()
    print("Resources cleaned up successfully")
except Exception as e:
    print(f"Cleanup warning: {e}")

INFO:qwen3_rag:Reranker cleaned up successfully
INFO:qwen3_rag:Generator cleaned up successfully


Resources cleaned up successfully


## System Statistics and Configuration

In [14]:
# Display final statistics
stats = rag.get_stats()
print("Final System Statistics:")
print(f"Document count: {stats['document_count']}")
print(f"Configuration: {stats['config']}")

Final System Statistics:
Document count: 57
Configuration: RAGConfig(embedding_model='Qwen/Qwen3-Embedding-0.6B', reranker_model='Qwen/Qwen3-Reranker-0.6B', generator_model='Qwen/Qwen2.5-3B-Instruct', chunk_size=512, chunk_overlap=50, top_k_retrieval=20, top_k_rerank=5, similarity_threshold=0.3, device='cuda:0', collection_name='my_documents', persist_directory='./my_chroma_db', max_context_length=4000, generation_temperature=0.1, generation_max_tokens=2048)
